# 要約 
このJupyterノートブックは、Kaggleのコンペティション「LLM 20 Questions」に参加するためのエージェントを開発することを目的としています。具体的には、20の質問ゲームの質問者（質問を行うAI）と回答者（「はい」または「いいえ」で答えるAI）の役割を果たす2種類のエージェントを実装しています。

### 問題の概要
ノートブックは、質問者エージェントおよび回答者エージェントがどうにかしてターゲットワードを推測できるように設計されています。エージェントは、過去の質問と回答をもとに次の質問を生成したり、与えられた質問に適切に答えたりする必要があります。このゲームの制約に基づき、効率的な情報収集と推論が求められます。

### 使用されている手法とライブラリ
- **Gemmaモデル**: ノートブックでは、`gemma_pytorch`リポジトリからGemma（Causal Language Model）を使用しています。これにより、モデルは質問応答を行い、ターゲットワードの推測を助けます。
- **PyTorch**: モデルを構築し、深層学習の計算を行うために使用されています。GPUアクセラレーターを活用して効率的に処理を行います。
- **クラス設計**:
    - `GemmaFormatter`クラス: プロンプトのフォーマッティングを行います。ゲームのターンごとにユーザーとモデルの入力を管理します。
    - `GemmaAgent`クラス: ゲームのエージェントのベースクラスで、質問者エージェント(`GemmaQuestionerAgent`)と回答者エージェント(`GemmaAnswererAgent`)の機能が含まれます。これらのクラスは、ゲームのセッションを管理し、応答を生成する役割を担っています。
- **関数**: `agent_fn`は、観察オブジェクト（ゲームの状態）に基づいて、エージェントを呼び出し、適切な回答を得る関数です。
  
### 実装手順
1. 必要なライブラリのインストールとリポジトリのクローンを行う。
2. システムパスを設定し、モデルの重みをロードする。
3. `GemmaAgent`クラスのインスタンスを生成し、質問者として機能する場合と回答者として機能する場合の両方を設定できるようにする。
4. 20の質問ゲームのロジックを実装し、それをテストするための観察オブジェクトを準備する。

### 結論
このノートブックは、20の質問ゲームにおける質問生成および回答生成のための強化されたエージェントを開発するための基盤を提供し、Gemmaという言語モデルを利用して、効果的なゲームプレイを実現しようとしています。これは、自然言語処理と機械学習技術の実用的な応用例であり、特に制約のある環境での戦略的な応答生成に焦点を当てています。

---


# 用語概説 
このJupyter Notebook内で初心者がつまずきそうな専門用語や概念について、簡単に解説します。

1. **マジックコマンド**: Jupyter Notebook内で特別な機能を提供するコマンドで、通常のPythonコードとは異なる方法で実行されます。例えば、`%%bash`はBashシェルコマンドを実行するために使用されます。

2. **immutabledict**: Pythonの標準の辞書型ではなく、変更不可能な辞書の実装です。データの不変性が保証され、プログラムの安全性が向上します。特に、ハッシュ可能性が必要な場合に使用されます。

3. **sentencepiece**: 自然言語処理において、文章をトークン化するためのライブラリで、特にサブワードトークン化を使用します。これは、単語の部分を考慮することで、未知の単語にも対応可能になります。

4. **GemmaForCausalLM**: 特定の言語モデルであり、因果関係に基づいて次の単語を予測するためのものです。ここでは、質問応答のために設計されたモデルとして使用されています。

5. **contextlib**: Pythonの標準ライブラリで、コンテキストマネージャを作成するためのユーティリティを提供します。リソースを管理しやすくするために使用されます。

6. **dtype**: データ型（data type）を指します。PyTorchの`torch.dtype`は、テンソルが保持するデータの型を指定するのに使われます。例としては、整数型や浮動小数点型などがあります。

7. **量子化 (quantization)**: 深層学習モデルのサイズを縮小し、推論速度を向上させるために使用される技術です。モデルの重みを少ないビット数（通常は8ビット）で表現します。これによりメモリ使用量が削減されますが、精度の低下が懸念されます。

8. **トークナイザー**: テキストをトークン（単語やサブワード）に変換するコンポーネントです。モデルにデータを供給する際の前処理ステップとして重要です。

9. **ask, answer, guess**: これらは「ターンタイプ」と呼ばれ、エージェントが何をするかを示す状態を表します。それぞれ質問をする、質問の答えを出す、または答えを推測することを意味します。

10. **エージェント**: 特定のタスクを実行するために設計されたプログラムまたはモデルです。このノートブックでは、質問をするエージェント（質問者）と、答えを提供するエージェント（回答者）が登場します。

11. **プロンプト**: モデルに入力を与えるためのテキストや情報です。プロンプトの設計は、応答の質や関連性に大きな影響を与えます。

12. **few-shot examples**: モデルが少数の例から学習し、新たなタスクを遂行するための技術です。これにより、少量のデータからでも効果的にパフォーマンスを発揮できるようになります。

これらの用語や概念は、特に実務経験のない初心者にとっては直感的でないことがありますので、理解を深めるためにこれらの解説が役立つでしょう。

---


In [ ]:
"""
あなたの提出コードはここに記述する必要があります。 
提出コードと.pyファイルを書き出すためのマジックコマンドをコメントアウトしてください。
シミュレーションされたゲームの値を渡すことができるobsクラスを作成するコードブロックを末尾に追加します。
GPUアクセラレーターを有効にして実行してください。
参照のために、下記にスターターノートブックが追加されています。
"""

## スターターノートブックの例
テスターをより早く見るために、2bに変更しただけです。 


https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook?kernelSessionId=178755035

In [ ]:
%%bash
# 作業ディレクトリに移動します
cd /kaggle/working

# immutabledict と sentencepiece を指定したフォルダーにインストールします
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# gemma_pytorchリポジトリをクローンします
git clone https://github.com/google/gemma_pytorch.git > /dev/null

# gemma用の新しいディレクトリを作成します
mkdir /kaggle/working/submission/lib/gemma/

# gemma_pytorchのファイルを指定したディレクトリに移動します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
#%%writefile submission/main.py
# セットアップ
import os
import sys

# **重要:** システムパスをこのように設定し、ノートブックとシミュレーション環境の両方でコードが動作するようにします。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトのフォーマッティング
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# エージェント定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """デフォルトのtorchのdtypeを指定されたdtypeに設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化中")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を担います。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを推測してください。推測は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を担います。キーワードは {obs.keyword} で、カテゴリは {obs.category} です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"この質問はキーワード {obs.keyword} に関するものです。はいまたはいいえで答えてください。答えは二重アスタリスクで囲んでください、例: **yes** または **no**。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを思いつき、質問者がはいまたはいいえの質問をします。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を担います。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** では、キーワードを推測してください。",
    "**フランス**", "正解です！",
]


# **重要:** エージェントをグローバル変数として定義し、必要なエージェントだけをロードします。
# 両方をロードすると、OOMになる可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "はい"
    else:
        return response

提出ファイルの作成がコメントアウトされました

In [ ]:
# !apt install pigz pv > /dev/null
# pigzとpvをインストールしますが、出力は表示しません
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2
# 提出物のtar.gzアーカイブを作成します、圧縮はpigzを使用してスピードアップし、pvで進行状況を表示します

## クイックテスト

これは質問者と推測者エージェントに適用されます。

In [ ]:
class obs(object):
    '''
    このクラスはゲームからの観察を表します。 
    turnTypeは['ask','answer','guess']のいずれかである必要があります。
    テストしたいキーワードを選択してください。
    キーワードのカテゴリを選択してください。
    質問は文字列のリストにできます。エージェントに過去の質問の履歴を渡す場合に使用します。
    回答も文字列のリストにできます。
    responseを使用して質問を回答者エージェントに渡します。
    '''
    def __init__(self, turnType, keyword, category, questions, answers):
        self.turnType = turnType
        self.keyword = keyword
        self.category = category
        self.questions = questions
        self.answers = answers

#エージェントに渡すobsの例。ここに独自のobsを渡してください。
obs1 = obs(turnType='ask', 
          keyword='paris', 
          category='place', 
          questions=['これは人ですか？', 'これは場所ですか？', 'これは国ですか？', 'これは都市ですか？'], 
          answers=['いいえ', 'はい', 'いいえ', 'はい'])

question = agent_fn(obs1, None)
print(question)

#出力としてエージェントからの応答を得る必要があります。 
#これは提出エラーの助けにはなりませんが、エージェントの応答を評価するのには役立ちます。

In [ ]:
obs2 = obs(turnType='guess', 
          keyword='paris', 
          category='place', 
          questions=['これは人ですか？', 'これは場所ですか？', 'これは国ですか？', 'これは都市ですか？', 'キーワードは雨が多いですか？'], 
          answers=['いいえ', 'はい', 'いいえ', 'はい', 'いいえ'])

agent_fn(obs2, None)